### Fine-Tune Mistral 7b

To install Unsloth on your own computer, run the following below:

In [1]:
# %pip install unsloth
# Also get the latest nightly Unsloth!
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

References: 

https://github.com/unslothai/unsloth

https://docs.unsloth.ai/tutorials/how-to-finetune-llama-3-and-export-to-ollama#id-6.-alpaca-dataset

https://www.youtube.com/watch?v=Gpyukc6c0w8

https://huggingface.co/datasets/laion/OIG

https://mer.vin/2024/02/unsloth-fine-tuning/

RAG: https://www.youtube.com/watch?v=LKokLun3bHI

https://huggingface.co/blog/synthetic-data-save-costs

#### Load Model Through Unsloth API

In [2]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/simges/venvs/fine-tune/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
[2024-12-11 19:43:16,058] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


==((====))==  Unsloth 2024.12.4: Fast Mistral patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA Graphics Device. Max memory: 15.702 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


- Define format and function in order to format dataset,
- It is supposed to include "text" keyword as shown below:

In [3]:
prompt = """ [ {"role": "user", "content": {}}, {"role": "assistant", "content": {}}] """

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def format_prompts(batch):
    answers         = batch["answer"]
    questions       = batch["question"]
    texts = []
    for answer, question in zip(answers, questions):
        conversation = []
        conversation.append({"role": "user", "content": question})
        conversation.append({"role": "assistant", "content": answer})
        chat = ','.join(str(x) for x in conversation) + EOS_TOKEN
        texts.append(chat)
    return { "text" : texts, }

def format_pandora_prompt(batch):
    messages = batch["messages"]
    texts = []
    for message in messages:
        chat = ','.join(str(item) for item in message) + EOS_TOKEN
        texts.append(chat)

    return { "text" : texts, }


- Load dataset and format the prompts

In [4]:
from datasets import load_dataset
dataset_name = "danilopeixoto/pandora-tool-calling"

dataset = load_dataset(dataset_name, split="train")
dataset = dataset.map(format_pandora_prompt, batched=True)

# Print a sample to verify
print(str(len(dataset['text'])))
print(dataset['text'][9])

101114
{'role': 'system', 'content': 'You are a helpful assistant with access to functions and tools. Use them if required.', 'tools': None, 'tool_calls': None, 'name': None},{'role': 'system', 'content': None, 'tools': '[{"name": "check_email_availability", "description": "Check if an email address is available", "parameters": {"type": "object", "properties": {"email": {"type": "string", "description": "The email address to be checked"}}, "required": ["email"]}}]', 'tool_calls': None, 'name': None},{'role': 'user', 'content': 'Hi, I want to create a new email account. Can you check if the email address "johnsmith@gmail.com" is available?', 'tools': None, 'tool_calls': None, 'name': None},{'role': 'assistant', 'content': None, 'tools': None, 'tool_calls': '[{"name": "check_email_availability", "arguments": {"email": "johnsmith@gmail.com"}}]', 'name': None},{'role': 'tool', 'content': '{"status": "unavailable", "message": "The email address \'johnsmith@gmail.com\' is already in use."}',

- Import wandb to trace and log fine-tuning process,
- import trainer and set training parameters.

In [5]:
import wandb

wandb.login(key = "286a64856911076c58502477a4541c62b0deb138")
run = wandb.init(
    project='Fine tuning mistral 7B',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: simgesonmez90 (simgesonmez90-i-zmir-institute-of-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/simges/.netrc


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os,torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps=-1, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(), # lower precision for adam optimizer, 32 bit is costly
        bf16 = is_bfloat16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "tool_calling_llm",
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [9]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA Graphics Device. Max memory = 15.702 GB.
4.441 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 101,114 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 12,639
 "-____-"     Number of trainable parameters = 20,971,520
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  0%|          | 50/12639 [05:28<23:28:19,  6.71s/it]

{'loss': 0.3723, 'grad_norm': 0.4719901382923126, 'learning_rate': 0.00019928763653633054, 'epoch': 0.0}


  1%|          | 100/12639 [10:52<20:09:27,  5.79s/it]

{'loss': 0.2893, 'grad_norm': 0.4467845559120178, 'learning_rate': 0.0001984961215766978, 'epoch': 0.01}


  1%|          | 150/12639 [15:10<20:17:16,  5.85s/it]

...

{'loss': 0.2527, 'grad_norm': 0.2999191880226135, 'learning_rate': 1.408896628146272e-06, 'epoch': 0.99}


100%|█████████▉| 12600/12639 [18:12:15<03:44,  5.75s/it]

{'loss': 0.2439, 'grad_norm': 0.41267263889312744, 'learning_rate': 6.173816685135349e-07, 'epoch': 1.0}


100%|██████████| 12639/12639 [18:15:51<00:00,  5.20s/it]

{'train_runtime': 65751.5389, 'train_samples_per_second': 1.538, 'train_steps_per_second': 0.192, 'train_loss': 0.2648867760214665, 'epoch': 1.0}


In [1]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'torch' is not defined

In [2]:
%pip install ipywidgets
model.save_pretrained_merged("my-mistral", tokenizer, save_method = "merged_16bit",)

Note: you may need to restart the kernel to use updated packages.


NameError: name 'model' is not defined